In [27]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch 
from tqdm import tqdm

In [29]:
import json
from datasets import load_dataset

# get instances
dataset = load_dataset(
    'wiki_auto', 'auto_acl')

Reusing dataset wiki_auto (/Users/garylai/.cache/huggingface/datasets/wiki_auto/auto_acl/1.0.0/5ffdd9fc62422d29bd02675fb9606f77c1251ee17169ac10b143ce07ef2f4db8)
100%|██████████| 1/1 [00:00<00:00, 176.28it/s]


In [30]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/stsb-roberta-large')

In [31]:
# def get_score(normal_sentence, simple_sentence):
#     return model.predict([normal_sentence, sample['simple_sentence'])])

In [32]:
scores = []
for sample in tqdm(zip(dataset['full'][:50]['normal_sentence'], dataset['full'][:50]['simple_sentence'])):
    normal_sentence, simple_sentence = sample
    score = model.predict([normal_sentence, simple_sentence])
    scores.append(score)

50it [00:19,  2.58it/s]


In [36]:
score_tensor = torch.tensor(scores); score_tensor

tensor([0.7096, 0.8300, 0.6466, 0.9433, 0.7912, 0.7776, 0.8060, 0.9277, 0.9156,
        0.6349, 0.8082, 0.6108, 0.7853, 0.7917, 0.8554, 0.7975, 0.9674, 0.7456,
        0.8347, 0.5539, 0.6691, 0.7091, 0.8786, 0.8974, 0.5886, 0.5590, 0.8940,
        0.6805, 0.8174, 0.7294, 0.8589, 0.9721, 0.8991, 0.7420, 0.8947, 0.9384,
        0.8461, 0.6881, 0.8158, 0.7774, 0.7708, 0.9630, 0.6442, 0.7259, 0.4262,
        0.7064, 0.8131, 0.6779, 0.6815, 0.8355])

In [51]:
topk_scores, topk_indices = torch.topk(score_tensor, 50)

In [88]:
index_to_score = {}
for sample in zip(topk_indices, topk_scores):
    index, score = sample
    index_to_score[int(index)] = float(score)

index_to_score

{31: 0.9721257090568542,
 16: 0.9674012660980225,
 41: 0.9630268216133118,
 3: 0.9432664513587952,
 35: 0.9384129047393799,
 7: 0.9276638031005859,
 8: 0.9155656099319458,
 32: 0.8990707397460938,
 23: 0.8974476456642151,
 34: 0.8946980834007263,
 26: 0.893998920917511,
 22: 0.8785606026649475,
 30: 0.8589243292808533,
 14: 0.8553726673126221,
 36: 0.8461111187934875,
 49: 0.8355098962783813,
 18: 0.8347379565238953,
 1: 0.8300175666809082,
 28: 0.817444384098053,
 38: 0.8157970905303955,
 46: 0.8130649924278259,
 10: 0.8082419037818909,
 6: 0.8059822916984558,
 15: 0.7974823713302612,
 13: 0.7916831374168396,
 4: 0.791235089302063,
 12: 0.7853291630744934,
 5: 0.7775927782058716,
 39: 0.7774439454078674,
 40: 0.770824134349823,
 17: 0.7455523610115051,
 33: 0.7420403957366943,
 29: 0.7293545603752136,
 43: 0.7259481549263,
 0: 0.7095543146133423,
 21: 0.7090914845466614,
 45: 0.7063785791397095,
 37: 0.6881066560745239,
 48: 0.6814740896224976,
 27: 0.6804964542388916,
 47: 0.67789846

In [89]:
def select_longer_normal(difference=100):
    # to be selected, normal sentence must be at least 10 characters longer
    longer_normal = []
    for i in topk_indices:
        index = int(i)
        if (len(dataset['full'][index]['normal_sentence']) - len(dataset['full'][index]['simple_sentence'])) > difference:
            longer_normal.append(index)
    return longer_normal

In [90]:
longer_normal = select_longer_normal()
print(longer_normal)
print(len(longer_normal))

[35, 13, 43, 0, 20, 25]
6


In [91]:
filtered_ds = dataset['full'].select(longer_normal)

In [95]:
i = 43
normal_sentence, simple_sentence = dataset['full'][i]['normal_sentence'], dataset['full'][i]['simple_sentence']
score = model.predict([normal_sentence, simple_sentence]); print(f"score: {score}")

score: 0.7259481549263


In [92]:
for i, sample in enumerate(zip(filtered_ds['normal_sentence'], filtered_ds['simple_sentence'])):
    normal_sentence, simple_sentence = sample
    print("-" * 80)
    print(f"normal_sentence: {normal_sentence} \nsimple_sentence: {simple_sentence} \noriginal_index: {longer_normal[i]} \n" )
    print("score: ", index_to_score[longer_normal[i]])


--------------------------------------------------------------------------------
normal_sentence: A tank car -LRB- International Union of Railways -LRB- UIC -RRB- : tank wagon -RRB- is a type of railroad car -LRB- UIC : railway car -RRB- or rolling stock designed to transport liquid and gaseous commodities .
 
simple_sentence: A tank car or tank wagon is a type of railroad car designed to transport liquids or gases .
 
original_index: 35 

score:  0.9384129047393799
--------------------------------------------------------------------------------
normal_sentence: The Society for Worldwide Interbank Financial Telecommunication -LRB- SWIFT -RRB- , legally S.W.I.F.T. SCRL , provides a network that enables financial institutions worldwide to send and receive information about financial transactions in a secure , standardized and reliable environment .
 
simple_sentence: The Society for Worldwide Interbank Financial Telecommunication -LRB- SWIFT -RRB- is an international communication networ